In [27]:
import pandas as pd


dfNF = pd.read_csv('daily_average.csv')
dfWF = pd.read_csv('daily_averageNoFire.csv')

dfNFSD = pd.read_csv('NaturalFiresTOPOGRAPHY.csv')
dfWFSD = pd.read_csv('LocationsWFire.csv')

In [ ]:
print(dfNF.columns)
print(dfWF.columns)

temp = 28_100
hum = 7_28

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

columns_to_drop = ['day', 'index', 'Unnamed: 0', 'index.1']

# Drop the columns
df = dfNF.drop(columns=columns_to_drop)

correlation_matrix = df.corr()

plt.figure(figsize=(30, 26))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm')


plt.show()




In [ ]:
columns_to_keep = ['location', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.precipitation', 'hourly.wind_speed_10m', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.direct_normal_irradiance']
df = dfNF[columns_to_keep]
dfW = dfWF[columns_to_keep]
print(df.columns)
print(dfW.columns)

In [ ]:
columns_to_keep = ['location', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.precipitation', 'hourly.wind_speed_10m', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.direct_normal_irradiance']
df = dfNF[columns_to_keep]
dfW = dfWF[columns_to_keep]

df['wildfire'] = 1

df.set_index('location', inplace=True)

columns_days = {
    'hourly.temperature_2m': 8,
    'hourly.relative_humidity_2m': 8,
    'hourly.precipitation': 16,
    'hourly.wind_speed_10m': 16,
    'hourly.soil_temperature_28_to_100cm': 8,
    'hourly.soil_moisture_7_to_28cm': 8,
    'hourly.direct_normal_irradiance': 8
}

for column, days in columns_days.items():
    df = df.sort_index(ascending=False)
    df[f'{column}_lag'] = df.groupby(level=0)[column].shift(days)
    df = df.sort_index()

df.drop(columns=columns_days.keys(), inplace=True)

df.dropna(inplace=True)

# Without fire

dfW['wildfire'] = 0
dfW.set_index('location', inplace=True)

for column, days in columns_days.items():
    dfW = dfW.sort_index(ascending=False)
    dfW[f'{column}_lag'] = dfW.groupby(level=0)[column].shift(days)
    dfW = dfW.sort_index()

dfW.drop(columns=columns_days.keys(), inplace=True)
dfW.dropna(inplace=True)

print(df.columns)
print(dfW.columns)

dfNFSD['location'] = range(1, len(dfNFSD) + 1)
dfWFSD['location'] = range(1, len(dfWFSD) + 1)

# Convert 'CLC_CODE' to integer in dfNFSD and dfWFSD
dfNFSD['CLC_CODE'] = dfNFSD['CLC_CODE'].astype(int)
dfWFSD['CLC_CODE'] = dfWFSD['CLC_CODE'].astype(int)

print(dfNFSD.head())
print(dfWFSD.head())

#print(df.head())
#print(dfW.head())scientificNames


variables_to_add = ['roughness', 'aspect', 'slope', 'CLC_CODE']

# For df
df = df.merge(dfNFSD[variables_to_add], left_index=True, right_on='location', how='left')

# For dfW
dfW = dfW.merge(dfWFSD[variables_to_add], left_index=True, right_on='location', how='left')

In [28]:
# Este é o correto a executar

columns_to_keep = ['location', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.precipitation', 'hourly.wind_speed_10m', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.direct_normal_irradiance']
variables_to_add = ['roughness', 'aspect', 'slope', 'CLC_CODE','scientificNames']

dfNFSD['location'] = range(1, len(dfNFSD) + 1)
dfWFSD['location'] = range(1, len(dfWFSD) + 1)

# Convert 'CLC_CODE' to integer in dfNFSD and dfWFSD
dfNFSD['CLC_CODE'] = dfNFSD['CLC_CODE'].astype(int)
dfWFSD['CLC_CODE'] = dfWFSD['CLC_CODE'].astype(int)

df = dfNF[columns_to_keep]
dfW = dfWF[columns_to_keep]

df['wildfire'] = 1
dfW['wildfire'] = 0

df.set_index('location', inplace=True)
dfW.set_index('location', inplace=True)

columns_days = {
    'hourly.temperature_2m': 8,
    'hourly.relative_humidity_2m': 8,
    'hourly.precipitation': 16,
    'hourly.wind_speed_10m': 16,
    'hourly.soil_temperature_28_to_100cm': 8,
    'hourly.soil_moisture_7_to_28cm': 8,
    'hourly.direct_normal_irradiance': 8
}

location_features = {}

for location in df.index.unique():
    location_features[location] = {}

    for column, days in columns_days.items():
        df.sort_index(ascending=True, inplace=True)
        last_days = df.loc[location][-days:]
        location_features[location][f'{column}_lag'] = last_days[column].values

    location_features[location]['wildfire'] = df.loc[location]['wildfire'].values[0]

for location in dfNFSD['location']:
    for variable in variables_to_add:
        location_features[location][variable] = dfNFSD.loc[dfNFSD['location'] == location, variable].values[0]

for location in dfW.index.unique():
    location_features[int(location)+1770] = {}

    for column, days in columns_days.items():
        dfW.sort_index(ascending=True, inplace=True)
        last_days = dfW.loc[location][-days:]
        location_features[int(location)+1770][f'{column}_lag'] = last_days[column].values

    location_features[int(location)+1770]['wildfire'] = dfW.loc[location]['wildfire'].values[0]

for location in dfWFSD['location']:
    for variable in variables_to_add:
        location_features[int(location)+1770][variable] = dfWFSD.loc[dfNFSD['location'] == location, variable].values[0]

/tmp/ipykernel_4185/407642764.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wildfire'] = 1
/tmp/ipykernel_4185/407642764.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfW['wildfire'] = 0


In [72]:
import json
import numpy as np

# Convert numpy arrays to lists and numpy int64 to int
for location in location_features:
    for feature in location_features[location]:
        if isinstance(location_features[location][feature], np.ndarray):
            location_features[location][feature] = location_features[location][feature].tolist()
        elif isinstance(location_features[location][feature], np.int64):
            location_features[location][feature] = int(location_features[location][feature])

# Save the dictionary into a JSON file
with open('location_features.json', 'w') as f:
    json.dump(location_features, f)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

dfx = pd.DataFrame(location_features).T

# Split the DataFrame into features (X) and target (y)
X = dfx.drop('wildfire', axis=1)
y = dfx['wildfire']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = clf.predict(X_test)

# Evaluate the model
print('Accuracy:', accuracy_score(y_test, y_pred))

ValueError: setting an array element with a sequence.

In [2]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

dfNF = pd.read_csv('daily_average.csv')
dfWF = pd.read_csv('daily_averageNoFire.csv')

dfNFSD = pd.read_csv('NaturalFiresTOPOGRAPHY_mean_elev.csv')
dfWFSD = pd.read_csv('LocationsWFire_mean_elev.csv')


columns_to_keep = ['location', 'hourly.temperature_2m', 'hourly.relative_humidity_2m', 'hourly.precipitation', 'hourly.wind_speed_10m', 'hourly.soil_temperature_28_to_100cm', 'hourly.soil_moisture_7_to_28cm', 'hourly.direct_normal_irradiance',
                   'fwix', 'ffmc', 'dmc', 'dc', 'isi', 'bui']
variables_to_add = ['roughness', 'aspect', 'slope', 'CLC_CODE','scientificNames', 'mean_elev']

dfNFSD['location'] = range(1, len(dfNFSD) + 1)
dfWFSD['location'] = range(1, len(dfWFSD) + 1)

# Convert 'CLC_CODE' to integer in dfNFSD and dfWFSD
dfNFSD['CLC_CODE'] = dfNFSD['CLC_CODE'].astype(int)
dfWFSD['CLC_CODE'] = dfWFSD['CLC_CODE'].astype(int)

df = dfNF[columns_to_keep]
dfW = dfWF[columns_to_keep]

df['wildfire'] = 1
dfW['wildfire'] = 0

df.set_index('location', inplace=True)
dfW.set_index('location', inplace=True)

columns_days = {
    'hourly.temperature_2m': 8,
    'hourly.relative_humidity_2m': 8,
    'hourly.precipitation': 16,
    'hourly.wind_speed_10m': 16,
    'hourly.soil_temperature_28_to_100cm': 8,
    'hourly.soil_moisture_7_to_28cm': 8,
    'hourly.direct_normal_irradiance': 8,
    'fwix': 8,
    'ffmc': 8,
    'dmc': 16,
    'dc': 16,
    'isi': 8,
}


# Assuming 'scientificNames' is in dfNFSD
dfNFSD['scientificNames'] = dfNFSD['scientificNames'].str.split('; ')

#print(dfNFSD['scientificNames'])

# Use MultiLabelBinarizer to perform one-hot encoding
mlb = MultiLabelBinarizer()
tree_species_encoded = mlb.fit_transform(dfNFSD['scientificNames'])
#print(tree_species_encoded)

# Convert the encoded tree species to a DataFrame
df_tree_species = pd.DataFrame(tree_species_encoded, columns=mlb.classes_, index=dfNFSD.index)

#print(df_tree_species)

# Concatenate the original DataFrame with the tree species DataFrame
dfNFSD = pd.concat([dfNFSD, df_tree_species], axis=1)

# Now you can drop the original 'scientificNames' column
dfNFSD = dfNFSD.drop('scientificNames', axis=1)


print(df_tree_species.columns.tolist())
print("-------------------------------------------------")
print(dfNFSD.columns)

variables_to_add.extend(df_tree_species.columns.tolist())

# Repeat the same process for dfWFSD
dfWFSD['scientificNames'] = dfWFSD['scientificNames'].str.split('; ')

mlb = MultiLabelBinarizer()
tree_species_encoded = mlb.fit_transform(dfWFSD['scientificNames'])

df_tree_species = pd.DataFrame(tree_species_encoded, columns=mlb.classes_, index=dfWFSD.index)

dfWFSD = pd.concat([dfWFSD, df_tree_species], axis=1)

dfWFSD = dfWFSD.drop('scientificNames', axis=1)

#print(dfNFSD.columns)

variables_to_add.extend(df_tree_species.columns.tolist())

variables_to_add = list(set(variables_to_add))

variables_to_add = [var for var in variables_to_add if var != '']

print(variables_to_add)

variables_to_add = [var for var in variables_to_add if var != 'scientificNames']

location_features = {}

for location in df.index.unique():
    location_features[location] = {}

    for column, days in columns_days.items():
        df.sort_index(ascending=True, inplace=True)
        last_days = df.loc[location][-days:]
        location_features[location][f'{column}_lag'] = last_days[column].values

    location_features[location]['wildfire'] = df.loc[location]['wildfire'].values[0]

for location in dfNFSD['location']:
    for variable in variables_to_add:
        location_features[location][variable] = dfNFSD.loc[dfNFSD['location'] == location, variable].values[0]
    #for species in dfNFSD.columns[dfNFSD.columns.str.startswith('scientificNames')]:
    #    location_features[location][species] = dfNFSD.loc[dfNFSD['location'] == location, species].values[0]

#for location in dfNFSD['location']:
#    for variable in variables_to_add:
#        location_features[location][variable] = dfNFSD.loc[dfNFSD['location'] == location, variable].values[0]

for location in dfW.index.unique():
    location_features[int(location)+1770] = {}

    for column, days in columns_days.items():
        dfW.sort_index(ascending=True, inplace=True)
        last_days = dfW.loc[location][-days:]
        location_features[int(location)+1770][f'{column}_lag'] = last_days[column].values

    location_features[int(location)+1770]['wildfire'] = dfW.loc[location]['wildfire'].values[0]

#for location in dfWFSD['location']:
#    for variable in variables_to_add:
#        location_features[int(location)+1770][variable] = dfWFSD.loc[dfNFSD['location'] == location, variable].values[0]

for location in dfWFSD['location']:
    for variable in variables_to_add:
        location_features[int(location)+1770][variable] = dfWFSD.loc[dfWFSD['location'] == location, variable].values[0]
    #for species in dfWFSD.columns[dfWFSD.columns.str.startswith('scientificNames')]:
    #    location_features[int(location)+1770][species] = dfWFSD.loc[dfWFSD['location'] == location, species].values[0]



# Initialize an empty DataFrame
dfx = pd.DataFrame()

# Loop over each location
for location, features in location_features.items():
    # Flatten the arrays and create new features
    for i, temp in enumerate(features['hourly.temperature_2m_lag']):
        dfx.loc[location, f'temperature_{i}'] = temp

    for i, humidity in enumerate(features['hourly.relative_humidity_2m_lag']):
        dfx.loc[location, f'humidity_{i}'] = humidity

    for i, precipitation in enumerate(features['hourly.precipitation_lag']):
        dfx.loc[location, f'precipitation_{i}'] = precipitation

    for i, wind_speed in enumerate(features['hourly.wind_speed_10m_lag']):
        dfx.loc[location, f'wind_speed_{i}'] = wind_speed

    for i, soil_temperature in enumerate(features['hourly.soil_temperature_28_to_100cm_lag']):
        dfx.loc[location, f'soil_temperature{i}'] = soil_temperature

    for i, soil_moisture in enumerate(features['hourly.soil_moisture_7_to_28cm_lag']):
        dfx.loc[location, f'soil_temperature{i}'] = soil_moisture

    for i, irradiance in enumerate(features['hourly.direct_normal_irradiance_lag']):
        dfx.loc[location, f'direct_normal_irradiance{i}'] = irradiance

    for i, fwix in enumerate(features['fwix_lag']):
        dfx.loc[location, f'fwix_{i}'] = fwix

    for i, ffmc in enumerate(features['ffmc_lag']):
        dfx.loc[location, f'ffmc_{i}'] = ffmc

    for i, dmc in enumerate(features['dmc_lag']):
        dfx.loc[location, f'dmc_{i}'] = dmc

    for i, dc in enumerate(features['dc_lag']):
        dfx.loc[location, f'dc_{i}'] = dc

    for i, isi in enumerate(features['isi_lag']):
        dfx.loc[location, f'isi_{i}'] = isi

    for variable in variables_to_add:
        dfx.loc[location, variable] = features[variable]

    dfx.loc[location, 'wildfire'] = features['wildfire']

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = dfx.drop('wildfire', axis=1)
print(X.columns)

y = dfx['wildfire']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

/tmp/ipykernel_3894/727955142.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wildfire'] = 1
/tmp/ipykernel_3894/727955142.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfW['wildfire'] = 0


['', 'Acacia Mill.', 'Castanea sativa Mill.', 'Ceratonia siliqua L.', "Eucalyptus L'Hér.", 'Magnoliopsida', 'Olea europaea L.', 'Oryza sativa L.', 'Pinales', 'Pinus pinaster Aiton', 'Pinus pinea L.', 'Quercus L.', 'Quercus rotundifolia Lam.', 'Quercus suber L.', 'Tracheophyta', 'Vitis vinifera L.']
-------------------------------------------------
Index(['year', 'date', 'district', 'municipality', 'parish', 'local',
       'latitude', 'longitude', 'cause', 'elevation', 'hourly.time',
       'hourly.temperature_2m', 'hourly.relative_humidity_2m',
       'hourly.dew_point_2m', 'hourly.apparent_temperature',
       'hourly.precipitation', 'hourly.rain', 'hourly.snowfall',
       'hourly.snow_depth', 'hourly.weather_code', 'hourly.pressure_msl',
       'hourly.surface_pressure', 'hourly.cloud_cover',
       'hourly.cloud_cover_low', 'hourly.cloud_cover_mid',
       'hourly.cloud_cover_high', 'hourly.et0_fao_evapotranspiration',
       'hourly.vapour_pressure_deficit', 'hourly.wind_speed_10

/tmp/ipykernel_3894/727955142.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfx.loc[location, f'dc_{i}'] = dc
/tmp/ipykernel_3894/727955142.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfx.loc[location, f'dc_{i}'] = dc
/tmp/ipykernel_3894/727955142.py:180: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe

Index(['temperature_0', 'temperature_1', 'temperature_2', 'temperature_3',
       'temperature_4', 'temperature_5', 'temperature_6', 'temperature_7',
       'humidity_0', 'humidity_1',
       ...
       'Quercus suber L.', 'Oryza sativa L.', 'mean_elev', 'Pinales',
       'Tracheophyta', 'slope', 'Castanea sativa Mill.', 'Acacia Mill.',
       'Quercus rotundifolia Lam.', 'Pinus pinaster Aiton'],
      dtype='object', length=140)
Accuracy: 0.980225988700565


In [3]:
print(X.columns)

Index(['temperature_0', 'temperature_1', 'temperature_2', 'temperature_3',
       'temperature_4', 'temperature_5', 'temperature_6', 'temperature_7',
       'humidity_0', 'humidity_1',
       ...
       'Quercus suber L.', 'Oryza sativa L.', 'mean_elev', 'Pinales',
       'Tracheophyta', 'slope', 'Castanea sativa Mill.', 'Acacia Mill.',
       'Quercus rotundifolia Lam.', 'Pinus pinaster Aiton'],
      dtype='object', length=140)


In [51]:
# Make probability predictions on the testing set
y_pred_proba = clf.predict_proba(X_test)

# The result is a 2D array where the first element of each entry is the probability of class 0 and the second is the probability of class 1
# We are interested in the probability of class 1 (wildfire), so we select the second column with [:, 1]
y_pred_proba = y_pred_proba[:, 1]

# Now y_pred_proba contains the predicted probabilities of a wildfire
print(y_pred_proba)

[0.81 0.76 0.85 0.04 0.02 0.11 0.01 0.09 0.91 0.98 0.03 0.08 0.06 0.07
 0.09 0.   0.99 0.91 0.   0.   0.16 0.73 0.93 0.02 0.   0.03 0.96 0.09
 0.17 0.94 1.   0.13 0.06 0.23 0.33 0.97 0.71 0.04 0.09 0.94 0.05 0.37
 0.93 0.99 0.98 0.85 0.9  0.81 0.03 0.9  0.04 0.05 0.75 0.76 0.58 0.99
 0.97 0.   0.94 0.04 0.06 0.01 0.99 0.05 0.16 0.09 0.95 0.22 0.73 0.17
 0.96 0.97 0.1  0.95 0.19 0.9  0.   0.18 0.05 0.89 0.95 0.93 0.19 0.94
 0.33 0.99 0.06 0.98 0.94 0.83 0.98 0.11 0.14 0.94 0.11 0.95 0.97 0.13
 0.12 0.01 1.   0.98 0.07 0.27 0.83 0.03 0.23 0.8  0.34 0.92 0.99 0.07
 0.9  0.82 0.04 0.03 0.92 1.   0.94 0.18 0.15 0.96 0.14 0.97 0.17 0.9
 1.   0.58 0.07 0.53 0.97 0.91 0.94 0.97 0.2  0.16 0.   0.88 0.06 0.03
 0.93 0.12 0.06 0.12 0.82 0.94 0.27 0.75 0.81 0.9  0.   0.99 0.94 0.99
 0.91 0.08 0.14 0.06 0.   1.   0.01 0.72 0.04 0.41 0.3  0.89 0.06 1.
 0.92 0.67 0.   0.02 0.12 0.16 0.87 0.04 0.26 0.96 0.98 0.96 0.01 0.05
 0.96 0.91 0.93 0.28 1.   0.97 0.15 0.07 0.99 0.92 0.07 0.99 0.26 0.68
 0.97 0.0

In [4]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('Precision:', precision)
print('Recall:', recall)
print('F1 Score:', f1)

Precision: 0.9920212765957447
Recall: 0.9713541666666666
F1 Score: 0.9815789473684211


In [8]:
# Count the number of wildfires in the test set
num_wildfires = y_test.sum()

print('Number of wildfires in the test set:', num_wildfires)

Number of wildfires in the test set: 384.0


In [49]:
pd.set_option('display.max_rows', None)


y_pred_proba = clf.predict_proba(X_test)

# Select the probabilities of class 1 (wildfire)
y_pred_proba = y_pred_proba[:, 1]

df_results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Predicted_Probability': y_pred_proba})

df_results['Correct'] = df_results['Actual'] == df_results['Predicted']

print(df_results)

      Actual  Predicted  Predicted_Probability  Correct
713      1.0        1.0                   0.81     True
280      1.0        1.0                   0.76     True
1457     1.0        1.0                   0.85     True
3505     0.0        0.0                   0.04     True
2195     0.0        0.0                   0.02     True
2081     0.0        0.0                   0.11     True
3209     0.0        0.0                   0.01     True
1789     0.0        0.0                   0.09     True
479      1.0        1.0                   0.91     True
1072     1.0        1.0                   0.98     True
2288     0.0        0.0                   0.03     True
2177     0.0        0.0                   0.08     True
2910     0.0        0.0                   0.06     True
2571     0.0        0.0                   0.07     True
3361     0.0        0.0                   0.09     True
2414     0.0        0.0                   0.00     True
1736     1.0        1.0                   0.99  